In [3]:
from dotenv import load_dotenv
load_dotenv()


True

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document

## Basic chain repsonse from input

In [14]:
model = ChatOpenAI(
    model = "gpt-3.5-turbo-1106",
    temperature=0.4
)

prompt = ChatPromptTemplate.from_template(
    """
    Answer the user question:
    Question : {input}
    """
)

chain = prompt | model

response = chain.invoke(
    {
        "input" : "What is LCEL?"
    }
)
print(response)

content='LCEL stands for "Lowest Common Endangered Language," which is a term used to refer to the least spoken and most at-risk languages in the world. These languages are often in danger of becoming extinct due to a variety of factors, such as globalization, urbanization, and government policies. Efforts are being made to document and preserve these languages in order to protect the cultural heritage they represent.' response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 23, 'total_tokens': 103}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None} id='run-9bc86522-5e6a-4245-812e-2449d9d2f3a8-0'


## Provide with some context to understand what we are asking

In [15]:
model = ChatOpenAI(
    model = "gpt-3.5-turbo-1106",
    temperature=0.4
)

prompt = ChatPromptTemplate.from_template(
    """
    Answer the user question:
    Context : LangChain Expression Language, or LCEL, is a declarative way to chain LangChain components. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production). To highlight a few of the reasons you might want to use LCEL:
    Question : {input}
    """
)

chain = prompt | model

response = chain.invoke(
    {
        "input" : "What is LCEL?"
    }
)
print(response)

content='LCEL stands for LangChain Expression Language, and it is a declarative way to chain LangChain components. It was designed to support putting prototypes into production without any code changes, from simple chains to complex chains with hundreds of steps.' response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 115, 'total_tokens': 162}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_21ca6db2bf', 'finish_reason': 'stop', 'logprobs': None} id='run-afcfd81b-f40c-4721-a5c6-c7decb8a176b-0'


## Dynamically input context through Document class

In [19]:
docA = Document(
    page_content="LangChain Expression Language, or LCEL, is a declarative way to chain LangChain components. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production). To highlight a few of the reasons you might want to use LCEL:"
)

model = ChatOpenAI(
    model = "gpt-3.5-turbo-1106",
    temperature=0.4
)

prompt = ChatPromptTemplate.from_template(
    """
    Answer the user question:
    Context : {context}
    Question : {input}
    """
)

chain = prompt | model

response = chain.invoke(
    {
        "input": "What is LCEL?",
        "context": [docA]    
    }
)

print(response)

content='LCEL stands for LangChain Expression Language, which is a declarative way to chain LangChain components. It was designed to support putting prototypes into production without any code changes, and can handle chains with hundreds of steps. It offers a variety of reasons for use, such as creating simple "prompt + LLM" chains to complex chains.' response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 121, 'total_tokens': 189}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_21ca6db2bf', 'finish_reason': 'stop', 'logprobs': None} id='run-df4d5cd1-5da0-4f99-8150-ca11e7bca9d9-0'


## Using create document stuff chain


In [20]:
docA = Document(
    page_content="LangChain Expression Language, or LCEL, is a declarative way to chain LangChain components. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production). To highlight a few of the reasons you might want to use LCEL:"
)

model = ChatOpenAI(
    model = "gpt-3.5-turbo-1106",
    temperature=0.4
)

prompt = ChatPromptTemplate.from_template(
    """
    Answer the user question:
    Context : {context}
    Question : {input}
    """
)

# chain = prompt | model
chain = create_stuff_documents_chain(
    llm = model,
    prompt=prompt
)


response = chain.invoke(
    {
        "input": "What is LCEL?",
        "context": [docA]    
    }
)

print(response)

LCEL stands for LangChain Expression Language, and it is a declarative way to chain LangChain components. It was designed to support putting prototypes in production, with no code changes, and can be used for simple or complex chains with hundreds of steps.


## Invoking a webloader, vector db  and retriever  to invoke documents

In [61]:
# Retrieve Data
def get_docs(url):
    loader = WebBaseLoader(url)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=20
    )

    splitDocs = text_splitter.split_documents(docs)

    return splitDocs

def create_db(docs):

    embedding = OpenAIEmbeddings()
    vectorStore = FAISS.from_documents(docs , embedding = embedding)
    return vectorStore

def create_chain(vectorStore):
    
    model = ChatOpenAI(
        model = "gpt-3.5-turbo-1106",
        temperature=0.4)

    prompt = ChatPromptTemplate.from_template(
        """
        Answer the user question and dont asnwer any question outside of the context, subtly say " I dont Know" :
        Context : {context}
        Question : {input}
        """
    )

    # chain = prompt | model
    document_chain = create_stuff_documents_chain(
        llm = model,
        prompt=prompt
    )

    #invoke the retriever
    retriever = vectorStore.as_retriever(search_kwargs = {"k" : 3})

    retrieval_chain = create_retrieval_chain(
        retriever,
        document_chain
    )
    
    return retrieval_chain


In [63]:
docs = get_docs('https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel')
vectorStore = create_db(docs)
chain = create_chain(vectorStore)


response = chain.invoke(
    {
        "input" : "What is Ramesh?"
    }
)

print(response["answer"])

I don't know.
